In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
from torchsummary import summary

from norse.torch.functional.lif import LIFParameters
import norse.torch as snn

from concrete import fhe

In [2]:
#device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#hyperparams
no_epochs = 2
batch_size = 32
learning_rate = 0.001

In [4]:
#transform the dataset into tensors normalized range [-1, 1]
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5),(0.5))     
        ])

In [6]:
#data sets downloading and reading
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform
                                        )

test_dataset = torchvision.datasets.MNIST(root='data',
                                        train=False,
                                        download=True,
                                        transform=transform
                                        )


In [7]:
# The Dataloaders - used to load data into application for use
train_loader = torch.utils.data.DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True
                            )

test_loader = torch.utils.data.DataLoader(test_dataset,
                                    batch_size=batch_size,
                                    shuffle=False
                            )

In [13]:
class ConvolutionalNeuralNet(nn.Module):
    def __init__(self, 
                 num_classes: int = 10, 
                 method: str = "super", 
                 dtype: torch.dtype = torch.float,
                 dt: float = 0.001):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=3)
        self.fc1 = nn.Linear(64 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.flatten = nn.Flatten()

    

    def forward(self, x):
        x= self.conv1(x)
        x = self.integrate_fire(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = x.flatten(1)
        x = torch.relu(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x
    
    def integrate_fire(x):
        print("fire")
        return x
    
summary(ConvolutionalNeuralNet(10), (1,28,28), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
            Conv2d-3             [-1, 64, 8, 8]          36,928
            Linear-4                   [-1, 64]         262,208
            Linear-5                   [-1, 10]             650
Total params: 318,602
Trainable params: 318,602
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.48
Params size (MB): 1.22
Estimated Total Size (MB): 1.70
----------------------------------------------------------------


In [15]:
net = ConvolutionalNeuralNet(10).to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [17]:
torch.manual_seed(42)

def train_one_epoch(net, optimizer, train_loader):
    # Cross Entropy loss for classification when not using a softmax layer in the network
    loss = nn.CrossEntropyLoss()

    net.train()
    avg_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = net(data)
        loss_net = loss(output, target.long())
        loss_net.backward()
        optimizer.step()
        avg_loss += loss_net.item()

    return avg_loss / len(train_loader)

# Create the tiny CNN with 10 output classes
N_EPOCHS = no_epochs

# Train the network with Adam, output the test set accuracy every epoch
losses_bits = []
optimizer = torch.optim.Adam(net.parameters())
for _ in tqdm(range(N_EPOCHS), desc="Training"):
    losses_bits.append(train_one_epoch(net, optimizer, train_loader))



Training: 100%|██████████| 2/2 [01:06<00:00, 33.17s/it]


In [18]:
def test_torch(net, test_loader):
    """Test the network: measure accuracy on the test set."""

    # Freeze normalization layers
    net.eval()

    # Determine the total number of samples in the dataset
    total_samples = len(test_loader.dataset)

    all_y_pred = np.zeros((total_samples,), dtype=np.int64)
    all_targets = np.zeros((total_samples,), dtype=np.int64)

    # Iterate over the batches
    idx = 0
    for data, target in test_loader:
        # Accumulate the ground truth labels
        endidx = idx + target.shape[0]
        all_targets[idx:endidx] = target.numpy()

        # Run forward and get the predicted class id
        output = net(data).argmax(1).detach().numpy()
        all_y_pred[idx:endidx] = output

        idx = endidx

    # Print out the accuracy as a percentage
    n_correct = np.sum(all_targets == all_y_pred)
    accuracy = n_correct / total_samples * 100
    print(f"Test accuracy: {accuracy:.2f}%")
    

# Call the test function
test_torch(net, test_loader)

Test accuracy: 98.73%
